In [1]:
# data science libraries
import os
import pandas as pd
# import cv2
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(1000)

# keras and tf
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers

# other tf imports
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# model imports
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler

# for compiling the model
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy as cce
from tensorflow.keras.metrics import CategoricalAccuracy as cca

# print validation statement
print("all resources loaded")

all resources loaded


In [2]:
df = pd.read_csv('data/hmnist_28_28_RGB.csv')
df

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
0,192,153,193,195,155,192,197,154,185,202,...,173,124,138,183,147,166,185,154,177,2
1,25,14,30,68,48,75,123,93,126,158,...,60,39,55,25,14,28,25,14,27,2
2,192,138,153,200,145,163,201,142,160,206,...,167,129,143,159,124,142,136,104,117,2
3,38,19,30,95,59,72,143,103,119,171,...,44,26,36,25,12,17,25,12,15,2
4,158,113,139,194,144,174,215,162,191,225,...,209,166,185,172,135,149,109,78,92,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10010,183,165,181,182,165,180,184,166,182,188,...,208,185,187,208,186,186,206,187,189,0
10011,2,3,1,38,33,32,121,104,103,132,...,96,79,76,24,23,21,3,4,1,0
10012,132,118,118,167,149,149,175,156,160,184,...,204,181,178,181,159,153,172,151,145,0
10013,160,124,146,164,131,152,167,127,146,169,...,185,162,167,184,157,166,185,162,172,0


In [8]:
pixels = df[[i for i in df.columns if i != 'label']]
pixels

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2342,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351
0,192,153,193,195,155,192,197,154,185,202,...,134,173,124,138,183,147,166,185,154,177
1,25,14,30,68,48,75,123,93,126,158,...,82,60,39,55,25,14,28,25,14,27
2,192,138,153,200,145,163,201,142,160,206,...,149,167,129,143,159,124,142,136,104,117
3,38,19,30,95,59,72,143,103,119,171,...,73,44,26,36,25,12,17,25,12,15
4,158,113,139,194,144,174,215,162,191,225,...,201,209,166,185,172,135,149,109,78,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10010,183,165,181,182,165,180,184,166,182,188,...,181,208,185,187,208,186,186,206,187,189
10011,2,3,1,38,33,32,121,104,103,132,...,129,96,79,76,24,23,21,3,4,1
10012,132,118,118,167,149,149,175,156,160,184,...,190,204,181,178,181,159,153,172,151,145
10013,160,124,146,164,131,152,167,127,146,169,...,165,185,162,167,184,157,166,185,162,172


In [9]:
labels = df['label']
labels

0        2
1        2
2        2
3        2
4        2
        ..
10010    0
10011    0
10012    0
10013    0
10014    6
Name: label, Length: 10015, dtype: int64

In [12]:
X = []

for i in range(len(pixels)):
    row = np.asarray(pixels.iloc[i])
    row = np.resize(row, (28,28,3))
    X.append(row)

X

[array([[[192, 153, 193],
         [195, 155, 192],
         [197, 154, 185],
         ...,
         [193, 144, 153],
         [197, 154, 163],
         [194, 152, 160]],
 
        [[197, 162, 202],
         [201, 160, 197],
         [202, 160, 193],
         ...,
         [194, 147, 149],
         [198, 158, 166],
         [192, 153, 167]],
 
        [[197, 155, 193],
         [198, 154, 185],
         [202, 156, 191],
         ...,
         [194, 149, 152],
         [199, 161, 177],
         [194, 156, 175]],
 
        ...,
 
        [[194, 157, 191],
         [206, 173, 206],
         [206, 165, 192],
         ...,
         [191, 155, 183],
         [193, 159, 187],
         [193, 163, 197]],
 
        [[187, 154, 185],
         [203, 171, 199],
         [202, 161, 183],
         ...,
         [178, 131, 148],
         [187, 152, 174],
         [190, 159, 189]],
 
        [[171, 136, 164],
         [191, 153, 179],
         [196, 155, 176],
         ...,
         [173, 124, 138],
  

In [16]:
X = np.asarray(X)
X.shape

(10015, 28, 28, 3)

In [76]:
y = []

for i in range(len(labels)):
    l = labels.iloc[i]
    y.append(l)

y = np.asarray(y)
y

array([2, 2, 2, ..., 0, 0, 6])

In [77]:
class_weight = cw.compute_class_weight(
    class_weight='balanced',
    classes=range(7),
    y=y
)

class_weight = dict(enumerate(class_weight))
class_weight

{0: 4.375273044997815,
 1: 2.78349082823791,
 2: 1.301832835044846,
 3: 12.440993788819876,
 4: 0.21338020666879728,
 5: 10.075452716297788,
 6: 1.2854575792581184}

In [64]:
# from tensorflow.keras.utils import to_categorical

# y = to_categorical(y,7)
# type(y)

numpy.ndarray

In [78]:
y.shape

(10015,)

In [79]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [80]:
# print(X_train[0:5], X_test[0:5], y_train[0:5], y_test[0:5])
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(8012, 28, 28, 3) (2003, 28, 28, 3) (8012,) (2003,)


In [104]:
def seq_model(out=7):
    # sequential attempt
    inp =  Input((28, 28, 3))
    x = tf.keras.layers.Normalization()(inp)

    x = Conv2D(256, kernel_size=5, activation='relu', padding='same', kernel_initializer='glorot_normal')(x)
    x = Conv2D(256, kernel_size=5, activation='relu', padding='same', kernel_initializer='glorot_normal')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = Conv2D(128, kernel_size=3, activation='relu', padding='same', kernel_initializer='glorot_normal')(x) #, kernel_initializer='glorot_normal')(x)
    x = Conv2D(128, kernel_size=3, activation='relu', padding='same', kernel_initializer='glorot_normal')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = Conv2D(64, kernel_size=2, activation='relu', padding='same', kernel_initializer='glorot_normal')(x) #, kernel_initializer='glorot_normal')(x)
    x = Conv2D(64, kernel_size=2, activation='relu', padding='same', kernel_initializer='glorot_normal')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = Dropout(0.5)(x)

    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    out = Dense(out, activation='softmax')(x)

    model = Model(inp, out)

    print(model.summary())
    return model

In [105]:
model = seq_model(7)

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 28, 28, 3)]       0         
                                                                 
 normalization_5 (Normalizat  (None, 28, 28, 3)        7         
 ion)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 24, 24, 256)       19456     
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 12, 12, 256)      0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 10, 10, 128)       295040    
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 5, 5, 128)        0   

In [106]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.losses import SparseCategoricalCrossentropy as scce
from tensorflow.keras.metrics import SparseCategoricalAccuracy as scca

optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0)
loss=scce()
acc=scca()

In [107]:
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=[acc]
)

In [108]:
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=4, verbose=1,
    mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.000000001
)

In [109]:
model.fit(
    X_train, y_train,
    # validation_split=0.2,
    validation_data=(X_test, y_test),
    epochs=50,
    verbose=1,
    callbacks=[lr_reduction],
    batch_size=32,
    # class_weight=class_weight
)

Epoch 1/50
251/251 [==============================] - 13s 49ms/step - loss: 1.4944 - sparse_categorical_accuracy: 0.6186 - val_loss: 1.0135 - val_sparse_categorical_accuracy: 0.6690 - lr: 1.0000e-04
Epoch 2/50
251/251 [==============================] - 12s 50ms/step - loss: 1.0018 - sparse_categorical_accuracy: 0.6655 - val_loss: 0.9506 - val_sparse_categorical_accuracy: 0.6680 - lr: 1.0000e-04
Epoch 3/50
251/251 [==============================] - 12s 47ms/step - loss: 0.9923 - sparse_categorical_accuracy: 0.6709 - val_loss: 0.9539 - val_sparse_categorical_accuracy: 0.6675 - lr: 1.0000e-04
Epoch 4/50
251/251 [==============================] - 12s 48ms/step - loss: 0.9507 - sparse_categorical_accuracy: 0.6694 - val_loss: 0.9512 - val_sparse_categorical_accuracy: 0.6565 - lr: 1.0000e-04
Epoch 5/50
251/251 [==============================] - 12s 47ms/step - loss: 0.9344 - sparse_categorical_accuracy: 0.6726 - val_loss: 0.9091 - val_sparse_categorical_accuracy: 0.6595 - lr: 1.0000e-04
Epoch

In [88]:
model.save('model')

INFO:tensorflow:Assets written to: model/assets


In [97]:
# from tensorflow.keras.applications.resnet50 import ResNet50

# def resnet(out=7):
#     model = ResNet50(weights='imagenet',include_top=False)
#     x = model.output
#     x = GlobalAveragePooling2D()(x)
#     x = Dense(1024, activation='relu')(x)
#     x = Dense(128, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     out = Dense(out)(x)

#     for layer in model.layers[0:10]:
#         layer.trainable = False

#     model_final = Model(model.input, out)

#     # print(model_final.summary())
#     return model_final

In [98]:
# resnet_model = resnet(7)

In [99]:
# from tensorflow.keras.optimizers import SGD, Adam, RMSprop
# from tensorflow.keras.losses import SparseCategoricalCrossentropy as scce
# from tensorflow.keras.metrics import SparseCategoricalAccuracy as scca

# optimizer=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0)
# loss=scce()
# acc=scca()

In [100]:
# resnet_model.compile(
#     optimizer=optimizer,
#     loss=loss,
#     metrics=[acc]
# )

In [ ]:
# resnet_model.fit(
#     X_train, y_train,
#     # validation_split=0.2,
#     validation_data=(X_test, y_test),
#     epochs=15,
#     verbose=1,
#     callbacks=[lr_reduction],
#     batch_size=32,
#     # class_weight=class_weight
# )

In [111]:
from tensorflow.keras.models import load_model

load_model = load_model('model/model.h5')
load_model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 28, 28, 3)]       0         
                                                                 
 normalization_3 (Normalizat  (None, 28, 28, 3)        7         
 ion)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 28, 28, 256)       19456     
                                                                 
 conv2d_10 (Conv2D)          (None, 28, 28, 256)       1638656   
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 14, 14, 256)      0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 14, 14, 128)       2950